# Lecture 3-7 LAB: Multi-resolution Techniques

## 0.- Initialize filesystem and libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pywavelets
!pip install pydicom

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pydicom
import pywt

## 1.- Decomposing and reconstructing an image

### Level 1 decomposition and reconstruction

In [ ]:
# Load the image from Google Drive
image_path = '/content/drive/MyDrive/PIM/Images/IM000004.jpg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Apply 2D DWT to the image (single level decomposition)
coeffs2 = pywt.dwt2(image, 'db2')  # Using 'db2' wavelet
# coeffs contains an array (cA) and a tupla (cH1, cV1, cD1) (parenthesis are required)
cA1, (cH1, cV1, cD1) = coeffs2  # Approximation, horizontal, vertical, diagonal detail coefficients
# Alternatively:
# cA1 = coeffs2[0] # extracts the approximation coefficients cA1
# cH1, cV1, cD1 = coeffs2[1] # extracts the tupla of level 1 detail coefficients

# Reconstruct the image from the coefficients
reconstructed_image = pywt.idwt2(coeffs2, 'db2')

# Plotting the results
plt.figure(figsize=(8, 10))

# Original image
plt.subplot(3, 2, 1)
plt.imshow(image, cmap='gray', aspect='auto')
plt.title("Original image")

# Reconstructed image
plt.subplot(3, 2, 2)
plt.imshow(reconstructed_image, cmap='gray', aspect='auto')
plt.title("Reconstructed image")

# Approximation coefficients
plt.subplot(3, 2, 3)
plt.imshow(cA1, cmap='gray', aspect='auto')
plt.title("Approximation coefficients (cA1 or LL1)")

# Horizontal detail coefficients
plt.subplot(3, 2, 4)
plt.imshow(cH1, cmap='gray', aspect='auto')
plt.title("Horizontal detail coefficients (cH1 or LH1)")

# Vertical detail coefficients
plt.subplot(3, 2, 5)
plt.imshow(cV1, cmap='gray', aspect='auto')
plt.title("Vertical detail coefficients (cV1 or HL1)")

# Diagonal detail coefficients
plt.subplot(3, 2, 6)
plt.imshow(cD1, cmap='gray', aspect='auto')
plt.title("Diagonal detail coefficients (cD1 or HH1)")

plt.tight_layout()
plt.show()

### Level 2 decomposition and reconstruction

In [ ]:
# Load the image from Google Drive
image_path = '/content/drive/MyDrive/PIM/Images/IM000004.jpg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Perform a 2-level DWT decomposition using the 'db2' wavelet
coeffs2 = pywt.wavedec2(image, 'db2', level=2)

# Extract the coefficients: unpack 'coeffs2': the returned value is a list where
# the first element is the approximation coefficients and the remaining elements
# are tuples (cHi, cVi, cDi) containing the detail coefficients for each level
cA2, (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs2
# Alternatively:
# cA2 = coeffs2[0]                            # Approximation coefficients at level 2
# (cH2, cV2, cD2) = coeffs2[1] = coeffs2[-2]  # Detail coefficients at level 2 (H, V, D)
# (cH1, cV1, cD1) = coeffs2[2] = coeffs2[-1]  # Detail coefficients at level 1 (H, V, D)

# Reconstruct the image from the coefficients
reconstructed_image = pywt.waverec2(coeffs2, 'db2')

# Create a figure using subplot
plt.figure(figsize=(10, 10))

# Approximation coefficients (cA2)
plt.subplot(4, 4, 1)
plt.imshow(cA2, cmap='gray')
plt.title('LL2')
plt.axis('off')

# Horizontal detail coefficients (cH2)
plt.subplot(4, 4, 2)
plt.imshow(cH2, cmap='gray')
plt.title('LH2')
plt.axis('off')

# Vertical detail coefficients (cV2)
plt.subplot(4, 4, 5)
plt.imshow(cV2, cmap='gray')
plt.title('HL2')
plt.axis('off')

# Diagonal detail coefficients (cD2)
plt.subplot(4, 4, 6)
plt.imshow(cD2, cmap='gray')
plt.title('HH2')
plt.axis('off')

# Horizontal detail coefficients (cH1)
plt.subplot(2, 2, 2)
plt.imshow(cH1, cmap='gray')
plt.title('LH1')
plt.axis('off')

# Vertical detail coefficients (cV1)
plt.subplot(2, 2, 3)
plt.imshow(cV1, cmap='gray')
plt.title('HL1')
plt.axis('off')

# Diagonal detail coefficients (cD1)
plt.subplot(2, 2, 4)
plt.imshow(cD1, cmap='gray')
plt.title('HH1')
plt.axis('off')

plt.tight_layout()
plt.show()

## 2.- Image denoising

### OpenCV library

In [ ]:
# Load the image
image = cv2.imread('/content/drive/MyDrive/PIM/Images/X-ray_1.jpeg', cv2.IMREAD_GRAYSCALE)

# Add Gaussian noise to the image
mean = 0
std_dev = 30  # Noise level
gaussian_noise = np.random.normal(mean, std_dev, image.shape)
noisy_image = image + gaussian_noise
noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8) # to prevent values
                                             # lower than 0 and higher than 255

# Perform the DWT on the noisy image
wavelet = 'db2'
coeffs = pywt.wavedec2(noisy_image, wavelet, level=2)


# SOFT THRESHOLDING

# Apply soft thresholding to the wavelet coefficients
def soft_threshold(data, threshold):
    return np.sign(data) * np.maximum(np.abs(data) - threshold, 0)

# Estimate the universal threshold (VisuShrink)
sigma = np.median(np.abs(coeffs[-1])) / 0.6745  # uses the highest scale (coeffs[-1])
threshold = sigma * np.sqrt(2 * np.log(noisy_image.size))


# DENOISING

# Keep the approximation coefficients unchanged
cA = coeffs[0]

# Apply soft thresholding to the detail coefficients for level 2
cH2_thresh = soft_threshold(coeffs[1][0], threshold)  # Horizontal detail coefficients
cV2_thresh = soft_threshold(coeffs[1][1], threshold)  # Vertical detail coefficients
cD2_thresh = soft_threshold(coeffs[1][2], threshold)  # Diagonal detail coefficients

# Apply soft thresholding to the detail coefficients for level 1
cH1_thresh = soft_threshold(coeffs[2][0], threshold)  # Horizontal detail coefficients
cV1_thresh = soft_threshold(coeffs[2][1], threshold)  # Vertical detail coefficients
cD1_thresh = soft_threshold(coeffs[2][2], threshold)  # Diagonal detail coefficients

# Form the coeffs_thresh list directly
coeffs_thresh = [
    cA,
    (cH2_thresh, cV2_thresh, cD2_thresh),
    (cH1_thresh, cV1_thresh, cD1_thresh)
]


# Reconstruct the image using the thresholded coefficients
denoised_image = pywt.waverec2(coeffs_thresh, wavelet)
denoised_image = np.clip(denoised_image, 0, 255).astype(np.uint8)


# Calculate SNR and MSE
def calculate_snr(image1, image2):
    noise = image1 - image2
    signal_power = np.mean(image1 ** 2)
    noise_power = np.mean(noise ** 2)

    # Calculate Signal-to-Noise Ratio in decibels (dB)
    snr = 10 * np.log10(signal_power / noise_power)

    # Calculate Mean Squared Error (MSE)
    mse = np.mean((image1 - image2) ** 2)

    return snr, mse

# Compare the noisy image and the denoised image with the original image
snr_noisy, mse_noisy = calculate_snr(image, noisy_image)
snr_denoised, mse_denoised = calculate_snr(image, denoised_image)


# Visualize original, noisy, and denoised images
plt.figure(figsize=(10, 3))

plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title("Original image")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(noisy_image, cmap='gray')
plt.title(f"Noisy image\n (SNR(dB): {snr_noisy:.2f}, MSE: {mse_noisy:.2f})")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(denoised_image, cmap='gray')
plt.title(f"Denoised image\n (SNR(dB): {snr_denoised:.2f}, MSE: {mse_denoised:.2f})")
plt.axis('off')

plt.tight_layout()
plt.show()

### Skimage library

In [ ]:
from skimage.restoration import estimate_sigma, denoise_wavelet
from skimage.util import random_noise
from skimage.metrics import peak_signal_noise_ratio as psnr

# Load the image
image = cv2.imread('/content/drive/MyDrive/PIM/Images/X-ray_1.jpeg', cv2.IMREAD_GRAYSCALE)

# Add Gaussian noise to the image using skimage's random_noise function
noisy_image = random_noise(image, mode='gaussian', mean=0, var=0.01)
noisy_image = np.clip(noisy_image * 255, 0, 255).astype(np.uint8)

# Estimate noise standard deviation using scikit-image's estimate_sigma
sigma_est = estimate_sigma(noisy_image)

# Perform wavelet denoising using scikit-image's denoise_wavelet
# - 'mode' can be 'hard' or 'soft' (thresholding)
# - 'method' can be 'VisuShrink' or 'BayesShrink'
# - 'rescale_sigma' adjusts the noise estimate (sigma) for wavelet-based scaling,
#   ensuring accurate thresholding based on the decomposition levels. It rescale
#   sigma based on the wavelet used. 'True' makes denoising more accurate by
#   adapting the noise estimate to each level
denoised_image = denoise_wavelet(noisy_image, sigma=sigma_est, method='VisuShrink',
                  wavelet='db2', mode='soft', wavelet_levels=2, rescale_sigma=True)

# Clip values to ensure they are within the valid range for an 8-bit image
denoised_image = np.clip(denoised_image * 255, 0, 255).astype(np.uint8)

# Calculate PSNR and MSE using skimage's psnr and custom formula
psnr_noisy = psnr(image, noisy_image)
psnr_denoised = psnr(image, denoised_image)

# Visualize original, noisy, and denoised images
plt.figure(figsize=(10, 3))

plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title("Original image")

plt.subplot(1, 3, 2)
plt.imshow(noisy_image, cmap='gray')
plt.title(f"Noisy image\n (PSNR(dB): {psnr_noisy:.2f})")

plt.subplot(1, 3, 3)
plt.imshow(denoised_image, cmap='gray')
plt.title(f"Denoised image\n (PSNR(dB): {psnr_denoised:.2f})")

plt.tight_layout()
plt.show()